In [46]:
#import libraries
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
# Import libraries to aid your analysis
from bokeh.plotting import figure, output_notebook, show #builds interactive graphs for python
from bokeh.models import Range1d
import math #this is used in graphic section to use the irrational number pi
output_notebook() #Allows inline plotting for Juptyer notebook


Loading BokehJS ...

In [47]:
# get events from n days ago
iso = 562 #Look in ACLED documentation ISO code for number this is for Niger
limit = 400

api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}'.format(iso)
print (api_url, type(api_url)) #creates request according to ACLED format specifications - p. 13

https://api.acleddata.com/acled/read?terms=accept&iso=562 <class 'str'>


In [48]:
response = requests.get(api_url)
data = response.json()
data.keys()
data['count'] 

500

### From the documentation we know this is the max return --- How can we get all the results?

In [24]:
# Let's mkae a function that updates our search to get the new pages

def ping_acled(api_url): 
    '''
    Takes one parameter search term for API
    '''
    
    response = requests.get(api_url)
    data = response.json()
    return data
    

In [25]:

results = [] # empty data strcture to store results
num_results = 500 # condition to continue adding pages
count = 0  # tracker of results
page = 1 #Per the documentation each page will give us more results

while num_results == 500: #if less 500 or 0 we know we have all the results
    print ("starting ", page, " ", num_results) #just to see our progress
    api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}&page={}'.format(iso,page) #the search
    data = ping_acled(api_url) #call the previous function 
    results.append(data['data']) #store in our results
    count += data['count'] #Track number of results
    num_results = data['count'] #update our condition
    page += 1 #update our page variable
    print ("Total Results ", count) #Track our progress
    



starting  1   500
Total Results  500
starting  2   500
Total Results  1000
starting  3   500
Total Results  1388


In [26]:
#Now I want to put them together into one giant result
super_list = []
for res in results: 
    super_list += res
    print (len(super_list))

500
1000
1388


In [27]:
#convert it into an pandas data frame or just use your data structure and do more stuff
niger_res = pd.DataFrame(super_list)
niger_res.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7202641,562,NIR1440,1440,2020-09-12,2020,1,Explosions/Remote violence,Air/drone strike,Military Forces of France (2017-),...,Garey Akoukou,14.7702,1.1053,2,Undisclosed Source,Local partner-Other,"On 12 September 2020, French Operation Barkhan...",10,1600112998,NER
1,7202167,562,NIR1439,1439,2020-09-12,2020,1,Battles,Armed clash,Unidentified Armed Group (Niger),...,Assamakka,19.3667,5.8000,2,Twitter; Undisclosed Source; Tadress24info; L'...,Local partner-Other,"On 12 September 2020, unknown gunmen attacked ...",2,1600112997,NER
2,7202611,562,NIR1438,1438,2020-09-10,2020,1,Battles,Armed clash,Unidentified Armed Group (Niger),...,Dao Timi,20.5575,13.5403,2,Tadress24info,New media,"On 10 September 2020, unknown gunmen and the N...",0,1600112998,NER
3,7202678,562,NIR1437,1437,2020-09-09,2020,1,Violence against civilians,Abduction/forced disappearance,Unidentified Armed Group (Niger),...,El Guidi,13.4580,6.6906,2,Whatsapp,New media,"On 9 September 2020, unknown gunmen abducted a...",0,1600112998,NER
4,7202677,562,NIR1434,1434,2020-09-08,2020,1,Battles,Armed clash,Boko Haram - Jamatu Ahli is-Sunnah lid-Dawatai...,...,Baroua Gana,13.8986,13.1544,2,Whatsapp,New media,"On 8 September 2020, Boko Haram (JAS) and ISWA...",10,1600112998,NER


### Do the right thing, take some time to look at the codebook and see what these columns are

In [8]:
niger_res.columns


Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

### Homework --- Make a map of some ACLED Data (absolutely use the code from the Global Terrorism Database exercise)

In [58]:
!pip install pyproj
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Transformer
title_provider = get_provider('STAMEN_TERRAIN')

In [72]:
country_map = niger_res[["latitude", 'longitude', 'actor1']]
country_map.head(7)

,latitude,longitude,actor1
0,14.7702,1.1053,Military Forces of France (2017-)
1,19.3667,5.8000,Unidentified Armed Group (Niger)
2,20.5575,13.5403,Unidentified Armed Group (Niger)
3,13.4580,6.6906,Unidentified Armed Group (Niger)
4,13.8986,13.1544,Boko Haram - Jamatu Ahli is-Sunnah lid-Dawatai...
5,13.5084,6.5903,Unidentified Armed Group (Niger)
6,13.2668,1.0284,JNIM: Group for Support of Islam and Muslims a...


In [71]:
pts = [(25.00,10.00), (0, 15.00)]
bbox = []
for pt in transformer.itransform(pts): 
    bbox.append(pt)

In [73]:
map_dict = {} # empty dictionary to track group attacks by lat long
nan_count = {} # some data doesn't have a lat/long so we need to know what we are losing


# Iterate through tables and associate group with lat/long
for idx, row in country_map.iterrows():    
    if row['actor1'] in map_dict.keys(): 
        if math.isnan(row["latitude"]):
            #This counts no data
            if row['actor1'] in nan_count.keys(): 
                nan_count[row['actor1']] += 1 
            else: 
                nan_count[row['actor1']] = 1
        else: 
            #This has to convert the lat/long to a mercator projection 
            point = transformer.transform(row["latitude"],row["longitude"])
            map_dict[row['actor1']].append([point[0],point[1]]) 
    #BOTH the if an else statement do the same thing but since it is a dictionary one needs to add the group name first
    else:  
        if math.isnan(row["latitude"]):
          nan_count[row['actor1']] = 1
        else: 
            point = transformer.transform(row["latitude"],row["longitude"])
            map_dict[row['actor1']] =[[point[0],point[1]]]
        
#This tells how many attacks we are losing
nan_count

TypeError: must be real number, not str

In [75]:
#Plots the bounding box
p = figure(x_range=(bbox[0][0], bbox[1][0]),y_range=(bbox[0][1], bbox[1][1]),x_axis_type="mercator", y_axis_type="mercator")
#add the map form the Bokeh map vendor in this case Stamen_Terrain --- see documentation
p.add_tile(tile_provider)
# Places a circle for each converted lat/long attack 
p.circle(x = NPA_x, y = NPA_y, color= "firebrick")

#shows the plot
show(p)

NameError: name 'tile_provider' is not defined